# Semi-Signed Distance Field for neural representation of open surfaces
This Jupyter Notebook serves as a way to display the results from training a neural network to represent geometric shapes.

You can visualize a 2D cross section of a particular shape and see how to access the latent vectors

#### Libraries

In [ ]:
# Geometric libraries
import igl
from pygel3d import hmesh, jupyter_display as jd
jd.set_export_mode(True)

# numpy libraries
import numpy as np
import math
import random
from numpy.linalg import norm
array = np.array
from scipy.interpolate import interp1d

# Deep learning libraries
import torch
import torch.nn as nn
import torch.nn.functional as Func
import torch.optim as optim
import torch.nn.init as init
from torch.nn.parameter import Parameter

import matplotlib.pyplot as plt


# Libraries for environmetn
import os
from dotenv import load_dotenv

In [ ]:
os.sys.path.insert(1,os.path.join(os.getcwd(),"..",".."))

#### External functions

In [ ]:
from utils.general import parse_cfg_file
from utils.neural_network import data
from utils.neural_network import Net   

In [ ]:
assert os.path.exists(os.path.join(os.getcwd(),"..","..",".env")), \
        "Please create an .env file with appropriate directories and specification of configuration file"
load_dotenv()
#--------------------------- 
# Environment
#---------------------------

# Data sets 
training_mesh_dir = os.path.join(os.getcwd(),os.getenv('mesh_train_dir'))
test_mesh_dir = os.path.join(os.getcwd(),os.getenv('mesh_test_dir'))
shape_completion_mesh_dir = os.path.join(os.getcwd(),os.getenv('mesh_shape_completion_dir'))

# Model dir
train_model_dir = os.path.join(os.getcwd(),os.getenv('ssdf_model_train_dir'))
test_model_dir = os.path.join(os.getcwd(),os.getenv('ssdf_model_test_dir'))
shape_completion_model_dir = os.path.join(os.getcwd(),os.getenv('ssdf_model_shape_completion_dir'))

# Interpolation dir
interpolation_dir = os.path.join(os.getcwd(),os.getenv('interpolation_dir'))
shape_reconsruction_dir = os.path.join(os.getcwd(),"shape_reconstruction")

# Configuration file
cfg_file = os.getenv('cfg_file')
assert os.path.exists(cfg_file), \
    "Make sure you have the config file (.yaml) in the cfgs folder and set correct path in .env file"
cfg = parse_cfg_file(cfg_file)

### Use GPU or CPU

In [ ]:
if (cfg.use_GPU):
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
else:
    device = 'cpu'

## Load data and network

#### Training data - Ground truth meshes and latent vectors

In [ ]:
training_data = data(training_mesh_dir, train_model_dir, cfg.model_train.name, device)
print("Number of training files:", len(training_data))

#### Test data - Ground truth meshes and latent vectors

In [ ]:
test_data = data(test_mesh_dir, test_model_dir, cfg.model_test.name, device)
print("Number of test files:", len(test_data))

#### Shape completion

In [ ]:
shape_completion_data = data(shape_completion_mesh_dir, shape_completion_model_dir, 
                             cfg.model_shape_completion.name, device, test_mesh_dir)
print("Number of training files:", len(shape_completion_data))

#### Neural Network

In [ ]:
net = Net(cfg.latent_vector.size + cfg.network.size_point, 
                cfg.network.num_hidden_units, cfg.network.multiple_output, device)

checkpoint = torch.load(os.path.join(train_model_dir,cfg.model_train.name),map_location=device)
net.load_state_dict(checkpoint['net_state_dict'])
net = net.to(device)
net.eval()

## Cross Sectional result

In [ ]:
'''
Training data
'''
#mesh, Faces, latent_vector = training_data["your_file.obj"] # Replace .obj file with name of own file

'''
Test data
'''
mesh, Faces, latent_vector = test_data["Pants_138.obj"]

In [ ]:
pos = mesh.positions()
epsilon = 0.1
bbox = hmesh.bbox(mesh)+epsilon*np.array([[-1,-1,-1],[1,1,1]])
resolution = 500
Y = np.linspace(bbox[0][1], bbox[1][1], resolution)
Z = np.linspace(bbox[0][2], bbox[1][2], resolution)
P = np.array([ np.array([0.0, Y[idx[0]], Z[idx[1]]]) for idx in np.ndindex((resolution,resolution))])

In [ ]:
UDF = abs(hmesh.MeshDistance(mesh).signed_distance(P))
UDF = UDF.reshape((resolution,resolution))
GWN = 2*np.array(igl.fast_winding_number_for_meshes(np.array(pos),np.array(Faces),np.array(P)))\
    -np.ones([1,resolution*resolution])
GWN = GWN.reshape((resolution,resolution))
SSDF = np.multiply(GWN, UDF)

In [ ]:
net_input = torch.cat((latent_vector.repeat(len(P),1).detach(),torch.from_numpy(P).float().to(device)),1)

net_output = net(net_input)
net_output_np = net_output.detach().numpy()

net_UDF = net_output_np[:,0]
net_UDF = net_UDF.reshape((resolution,resolution))

net_SSDF = net_output_np[:,1]
net_SSDF = net_SSDF.reshape((resolution,resolution))

net_GWN = np.true_divide(net_SSDF,net_UDF+1e-5)

In [ ]:
plt.figure()
f, axarr = plt.subplots(nrows=3, ncols=2, figsize=(20, 20), constrained_layout=True)

ax = axarr[0][0]
levels = [0]
cs = ax.contour(SSDF, levels,colors=['red'])
im3 = ax.imshow(SSDF, interpolation='bilinear', origin='lower')
ax.set_xlabel('y',fontsize=25)
ax.set_ylabel('z',fontsize=25)
f.colorbar(im3,ax = axarr[0][0])
ax.set_title("Ground Truth SSDF, x = 0.0", size=30)

ax = axarr[0][1]
levels = [0]
cs = ax.contour(net_SSDF, levels,colors=['red'] )
im3 = ax.imshow(net_SSDF, interpolation='bilinear', origin='lower')
ax.set_xlabel('y',fontsize=25)
ax.set_ylabel('z',fontsize=25)
f.colorbar(im3,ax = axarr[0][1])
ax.set_title("Network SSDF, x = 0.0", size=30)

ax = axarr[1][0]
levels = [0]
cs = ax.contour(UDF, levels)
im3 = ax.imshow(UDF, interpolation='bilinear', origin='lower')
ax.set_xlabel('y',fontsize=25)
ax.set_ylabel('z',fontsize=25)
f.colorbar(im3,ax = axarr[1][0])
ax.set_title("Ground Truth UDF, x = 0.0", size=30)

ax = axarr[1][1]
levels = [0]
cs = ax.contour(net_UDF, levels)
im3 = ax.imshow(net_UDF, interpolation='bilinear', origin='lower')
ax.set_xlabel('y',fontsize=25)
ax.set_ylabel('z',fontsize=25)
f.colorbar(im3,ax = axarr[1][1])
ax.set_title("Network UDF, x = 0.0", size=30)

ax = axarr[2][0]
levels = [-75, -70, -65, -60, -55, -50, -45, -40, -35, -30, -25, -20, -15, -10, -5, 0, 1, 2, 3, 4, 5, 6, 7, 8]
cs = ax.contour(GWN, levels)
im3 = ax.imshow(GWN, interpolation='bilinear', origin='lower')
ax.set_xlabel('y',fontsize=25)
ax.set_ylabel('z',fontsize=25)
f.colorbar(im3,ax = axarr[2][0])
ax.set_title("Ground Truth GWN, x = 0.0", size=30)

ax = axarr[2][1]
levels = [0]
cs = ax.contour(net_GWN, levels,colors=['red'])
im3 = ax.imshow(net_GWN, interpolation='bilinear', origin='lower')
ax.set_xlabel('y',fontsize=25)
ax.set_ylabel('z',fontsize=25)
f.colorbar(im3,ax = axarr[2][1])
ax.set_title("Network GWN, x = 0.0", size=30)

### Latent Vectors 

In [ ]:
training_data.latent_vectors.detach().cpu().numpy()

In [ ]:
test_data.latent_vectors.detach().cpu().numpy()

In [ ]:
shape_completion_data.latent_vectors.detach().cpu().numpy()